In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
reviews = pd.read_pickle("../../data/embedding_experiment/video_games/video_games_reviews_5core.p")
onehot = pd.read_pickle("../../data/embedding_experiment/video_games/video_games_onehot_5core.p")
sbert = pickle.load(open("../../data/embedding_experiment/video_games/video_games_sbert_5core.p", "rb"))

In [6]:
uuids, uiids = reviews["user_id"].unique(), reviews["asin"].unique()

In [7]:
user_map = dict(zip(uuids, list(range(1, len(uuids) + 1))))
item_map = dict(zip(uiids, list(range(1, len(uiids) + 1))))

In [12]:
pickle.dump(user_map, open("../../data/embedding_experiment/video_games/video_games_usermap_5core.p", "wb"))
pickle.dump(item_map, open("../../data/embedding_experiment/video_games/video_games_itemmap_5core.p", "wb"))

In [13]:
r_sorted = reviews.sort_values("timestamp")
r_sorted["iid"] = r_sorted["asin"].map(item_map)
r_sorted["uid"] = r_sorted["user_id"].map(user_map)

In [14]:
r_sorted["date"] = pd.to_datetime(r_sorted["timestamp"], unit="s")
r_sorted["year"], r_sorted["month"], r_sorted["day"], r_sorted["dayofweek"], r_sorted["dayofyear"], r_sorted["week"] = zip(*r_sorted["date"].map(lambda x: [x.year, x.month, x.day, x.dayofweek, x.dayofyear, x.week]))

norm_cols = ["year", "month", "day", "dayofweek", "dayofyear", "week"]
r_sorted[norm_cols] = r_sorted[norm_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [16]:
ctx = {}
for t in r_sorted.itertuples():
    ctx[(t.uid, t.iid)] = [t.year, t.month, t.day, t.dayofweek, t.dayofyear, t.week]

In [17]:
onehot["iid"] = onehot["asin"].map(item_map)
onehot_np = onehot.sort_values("iid").drop(["asin", "iid"], axis=1).to_numpy()

In [18]:
sbert_t = [(k, v) for k, v in sbert.items()]
sbert_df = pd.DataFrame(sbert_t, columns=["asin", "embedding"])
sbert_df["iid"] = sbert_df["asin"].map(item_map)
sbert_np = np.stack(sbert_df.sort_values("iid")["embedding"].to_numpy())

In [26]:
len(reviews)

526400

In [24]:
reviews.groupby("asin").count()

,user_id,timestamp
asin,,
0439381673,5,5
0439573947,1,1
0439591538,2,2
0439900581,1,1
0700026398,10,10
...,...,...
B01HIZF83S,1,1
B01HIZGKOE,6,6
B01HJ149LI,2,2


In [ ]:
reviews

In [19]:
with open("../../data/embedding_experiment/video_games/video_games_sorted_5core.txt", "w") as dst:
    for t in r_sorted.itertuples():
        uid = user_map[t.user_id]
        iid = item_map[t.asin]
        dst.write(f"{uid} {iid} {t.timestamp}\n")

In [20]:
with open("../../data/embedding_experiment/video_games/video_games_ctx_5core.dat", 'wb') as f:
    pickle.dump(ctx, f)

In [21]:
with open("../../data/embedding_experiment/video_games/video_games_onehot_5core.dat", 'wb') as f:
    pickle.dump(onehot_np, f)

In [22]:
with open("../../data/embedding_experiment/video_games/video_games_sbert_5core.dat", 'wb') as f:
    pickle.dump(sbert_np, f)